In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import struct
# import os
from pathlib import Path
from scipy.io import FortranFile
import pickle

plt.rcParams.update({'font.size': 16})


class DataProcessingClass:

    def __init__(self, DataPath, GridSize, Reduction):
        self.DataPath = DataPath
        self.GridSize = GridSize
        self.Reduction = Reduction
        self.HaloSize = int(self.GridSize/self.Reduction)
        self.HaloNumberTotal = Reduction**3
        self.CurrentPos = np.array([0,0,0])
        self.VirtualOriginLocal = np.array([0,0,0])
        self.VirtualOriginGlobal = np.array([0,0,0])
        self.GaussianBox = np.zeros([self.HaloSize,self.HaloSize,self.HaloSize])
        self.Cube = {}
        self.Parameters = 'u v w'.split()
        self.LESDictionary = {}
        self.LESPointDictionary = {}

    def Create_Data_Structure(self):

        """Creates Datastructure, Input fortran .Dat file"""

        data={}
        raw_dat = FortranFile(self.DataPath,mode='r')
        n = self.GridSize

        for index, parameter in enumerate(self.Parameters):
            data[parameter]=np.zeros((n,n,n))

        for i in range(n):
            for j in range(n):
                for k in range(n):
                    point_velocities = raw_dat.read_reals(dtype=np.dtype('f4'))
                    for index, parameter in enumerate(self.Parameters):
                        data[parameter][i][j][k]=point_velocities[index]

        self.data = data

        return 0


    def Plot_Scalar_2D_Data(self, pparams):

        """Does this and that"""

        AxesFound = np.array([0,0,0])

        if pparams[2] == 'x' or pparams[1] == 'x':
            AxesFound[0] = 1
        if pparams[2] == 'y' or pparams[1] == 'y':
            AxesFound[1] = 1
        if pparams[2] == 'z' or pparams[1] == 'z':
            AxesFound[2] = 1

        if sum(AxesFound) == 2:
            AxesFound *= self.GridSize
            AxesFound[list(AxesFound).index(0)] = pparams[3]
            PlotAxes = True
        else:
            PlotAxes = False
            print('Re write the executable 2D plot axes... Something went wrong')

        if PlotAxes:

            # mappingColorList = list(np.reshape(Data2D,PointNumber**2,1))
            plt.figure()
            plt.contourf(np.arange(self.GridSize),np.arange(self.GridSize),self.data[pparams[0]][0:AxesFound[0]][0:AxesFound[1]][0:AxesFound[2]][0])
            # plt.colorbar()  # mappable = mappingColorList
            plt.colorbar().set_label('Velocity in U direction (m/s)')
            # plt.title(('2D Representation of {} velocity at ({},{}) plane and height = {}').format(pparams[0],pparams[1],pparams[2],pparams[3]))
            plt.xlabel('Point in x')
            plt.ylabel('Point in y')
            plt.style.use('seaborn-paper')
            plt.show()

            return 0

    def HaloBox(self):
        # self.CurrentPos (IFSTATE)
        # Select Corner point based on where the current pointer is at...
        CornerPoint = self.CurrentPos
        for parameter in self.Parameters:
            self.Cube[parameter] = np.zeros((self.HaloSize,self.HaloSize,self.HaloSize))

        for parameter in self.Parameters:
            self.Cube[parameter] = (self.data[parameter]
                                  [CornerPoint[0]:CornerPoint[0]+self.HaloSize,
                                   CornerPoint[1]:CornerPoint[1]+self.HaloSize,
                                   CornerPoint[2]:CornerPoint[2]+self.HaloSize])
        return 0

    def translate(self,TranslateVector):
        self.CurrentPos += TranslateVector*self.HaloSize
        return 0

    def MVNCubeCreate(self,STDAddtionalMulti):
        GaussianBox = np.zeros([self.HaloSize,self.HaloSize,self.HaloSize])
        self.VirtualOriginLocal = 0.5*(self.HaloSize - 1)*np.array([1,1,1])
        self.VirtualOriginGlobal = self.VirtualOriginLocal + self.CurrentPos

        Multi = 0.5*(self.HaloSize - 1)/self.HaloSize
        StanDevXYZ = np.std(np.arange(0,self.HaloSize))*Multi*STDAddtionalMulti
        MeanVal = self.VirtualOriginLocal

        for z in range(0,self.HaloSize):
            for y in range(0,self.HaloSize):
                for x in range(0,self.HaloSize):
                    GaussianBox[x,y,z] = (1/(((2*np.pi)**1.5)*StanDevXYZ**3))*                        np.exp(-(1/(2*StanDevXYZ))*((x-MeanVal)**2+(y-MeanVal)**2+(z-MeanVal)**2))[0]

        K_d = np.sum(GaussianBox)**-1
        GaussianBox *= K_d  # KEEP COMMENTED FOR TRAINING PURPSES; COMMENT LATER
        self.GaussianBox = GaussianBox
        return 0

    def GaussianCubeCreate(self):
        FilterCube = np.zeros([self.HaloSize,self.HaloSize,self.HaloSize])
        self.VirtualOriginLocal = 0.5*(self.HaloSize - 1)*np.array([1,1,1])
        MeanVal = self.VirtualOriginLocal

        for z in range(0,self.HaloSize):
            for y in range(0,self.HaloSize):
                for x in range(0,self.HaloSize):
                    FilterCube[x,y,z] = (6/(np.pi*(self.HaloSize)**2))**1.5*                        np.exp((-6/(self.HaloSize**2))*((x-MeanVal)**2+(y-MeanVal)**2+(z-MeanVal)**2))[0]

        K_d = np.sum(FilterCube)**-1
        FilterCube *= K_d  # KEEP COMMENTED FOR TRAINING PURPSES; COMMENT LATER
        self.GaussianBox = FilterCube
        return 0

    def Gaussian2DPlot(self,index_y,index_z):
        plt.figure()
        plt.contourf(np.arange(self.HaloSize),np.arange(self.HaloSize),self.GaussianBox[:,:,index_z])
        plt.colorbar().set_label('Weighting')
        plt.xlabel('Point in x')
        plt.ylabel('Point in y')
        plt.style.use('seaborn-paper')
        plt.show()
        plt.figure()
        plt.plot(self.GaussianBox[:,index_y,index_z])
        plt.xlabel('Point in x at y = {} and z = {}'.format(index_y,index_z))
        plt.ylabel('Weighting')
        plt.show()
        return 0

    def Coarsening_Plot(self, plus, param):
        LES_vals = self.LESDictionary['MVV'][param][0][(plus)*self.Reduction**2:(plus+1)*self.Reduction**2]                .reshape(self.Reduction,self.Reduction)
        DNS_val_1 = self.data[param][:,:,plus + 1]
        DNS_val_2 = self.data[param][:,:,plus+ 2]
        DNS_val = 0.5*(DNS_val_1+DNS_val_2)

        plt.subplot(1,2,1)
        plt.contourf(np.arange(self.GridSize),np.arange(self.GridSize),DNS_val,cmap = 'jet')
        plt.xlabel('Point in x')
        plt.ylabel('Point in y')
        plt.colorbar().set_label('Velocity')

        #sns.heatmap(DNS_val,cmap='jet')
        plt.subplot(1,2,2)
        plt.contourf(np.arange(self.Reduction),np.arange(self.Reduction),LES_vals, cmap = 'jet')
        plt.colorbar().set_label('Mean Velocity')
        plt.xlabel('Point in x')
        plt.ylabel('Point in y')
        #sns.heatmap(LES_vals,cmap='jet')
        return 0

    def Coarsen_Plot_1D(self, plusy, param):
        LES_vals = self.LESDictionary['MVV'][param][0][(plusy)*self.Reduction:(plusy+1)*self.Reduction]
        DNS_vals = self.data[param][:,plusy,0]
        plt.plot(np.arange(0,self.GridSize,self.HaloSize),LES_vals, linewidth = 3, c = 'r', label = 'LES')
        plt.plot(np.arange(self.GridSize), DNS_vals, linewidth = 1.5, c = 'b', label = 'DNS')
        plt.xlabel('Point in X')
        plt.ylabel('Velocity (m/s)')
        plt.xlim([0,self.GridSize-self.HaloSize])
        plt.legend()
        return 0

    def LESDiction(self):
        LESSize = self.HaloNumberTotal
        DicMVV = {}
        DicMVG = {}
        DicIMV = {}
        DicMVI = {}
        cartesians = "x y z".split()

        for indexp, parameter in enumerate(self.Parameters):
            DicMVV[parameter] = np.zeros([1,LESSize])
            for indexc, cartesian in enumerate(cartesians):
                DicMVG[parameter+cartesian] = np.zeros([1,LESSize])

        for index1, parameter1 in enumerate(self.Parameters):
            for index2, parameter2 in enumerate(self.Parameters):
                DicIMV[parameter1+parameter2] = np.zeros([1,LESSize])
                DicMVI[parameter1+parameter2] = np.zeros([1,LESSize])

        self.LESPointDictionary = {
            "PN":np.zeros([1,LESSize]),
            "PV":np.zeros([1,3,LESSize]),
        }

        self.LESDictionary = {
            "MVV":DicMVV,
            "MVG":DicMVG,
            "IMV":DicIMV,
            "MVI":DicMVI
        }
        return 0

    def AppendPointLocation(self, PointNumb):
        self.LESPointDictionary['PN'][0,PointNumb] = PointNumb
        for index, params in enumerate(self.Parameters):
            self.LESPointDictionary['PV'][0,index,PointNumb] = self.CurrentPos[index]
        return 0

    def AppendHaloAverage(self, PointNumb, bGaussianFilter):
        if bGaussianFilter:
            for index, params in enumerate(self.Parameters):
                self.LESDictionary['MVV'][params][0,PointNumb] =                    np.tensordot(self.Cube[self.Parameters[index]],self.GaussianBox,axes=3)
        else:
            for index, params in enumerate(self.Parameters):
                self.LESDictionary['MVV'][params][0,PointNumb] =                    np.mean(self.Cube[self.Parameters[index]])
        return 0

    def AppendMVG(self, PointNumb, bGaussianFilter):
        cartesians = "x y z".split()
        if bGaussianFilter:
            for indexp, param in enumerate(self.Parameters):
                for indexc, cartesian in enumerate(cartesians):
                    self.LESDictionary['MVG'][param+cartesian][0, PointNumb] =                        np.tensordot(np.gradient(self.Cube[param],axis=indexc),self.GaussianBox,axes=3)
        else:
                    self.LESDictionary['MVG'][param+cartesian][0, PointNumb] =                        np.mean(np.gradient(self.Cube[param],axis=indexc))
        return 0

    def AppendIMV(self, PointNumb):
        for paramcol in self.Parameters:
            for paramrow in self.Parameters:
                self.LESDictionary['IMV'][paramrow + paramcol][0,PointNumb] =                    self.LESDictionary['MVV'][paramrow][0,PointNumb]*                    self.LESDictionary['MVV'][paramcol][0,PointNumb]
        return 0

    def AppendMVI(self, PointNumb, bGaussianFilter):
        for parami in self.Parameters:
            for paramj in self.Parameters:
                if bGaussianFilter:
                    self.LESDictionary['MVI'][parami + paramj][0,PointNumb] =                        np.tensordot(self.Cube[parami]*self.Cube[paramj],self.GaussianBox,axes=3)
                else:
                    self.LESDictionary['MVI'][parami + paramj][0,PointNumb] =                        np.mean(self.Cube[parami]*self.Cube[paramj])
        return 0

    def Coarsen(self):
        self.CurrentPos = np.array([0,0,0])
        TransVector = np.array([0,0,0])
        Coarsened = False
        PointNumber = 0

        for ziter in range(0,self.Reduction):
            for yiter in range(0,self.Reduction):
                for xiter in range(0,self.Reduction):

                    # Create a cube at the current position (self.CurrentPos)
                    self.HaloBox()

                    # Append Point location and Position
                    self.AppendPointLocation(PointNumber)

                    # Append Average velocity
                    self.AppendHaloAverage(PointNumber, True)  # Average Velocity vector at virtual point

                    # Append mean velocity gradient (MVG)
                    self.AppendMVG(PointNumber, True)

                    # Append interaction of mean velocities (IMV = mean(ui)*mean(uj))
                    self.AppendIMV(PointNumber)

                    # Append mean velocity interactions (MVI = mean(ui*uj))
                    self.AppendMVI(PointNumber, True)

                    # External gradients are done post-coarsening

                    if xiter == self.Reduction - 1:
                        if yiter != self.Reduction - 1:
                            TransVector = np.array([-(self.Reduction-1),1,0])
                        elif yiter == self.Reduction - 1 and ziter != self.Reduction - 1:
                            TransVector = np.array([-(self.Reduction-1),-(self.Reduction-1),1])
                        else:
                            print('Coarsening Complete')
                            Coarsened = True
                            TransVector = np.array([0,0,0])
                    else:
                        TransVector = np.array([1,0,0])
                    self.translate(TransVector)
                    PointNumber += 1
        return Coarsened

    def CreateDataFrame(self, detail = False):
        DatFrameLES = pd.DataFrame.from_dict({(i,j): self.LESDictionary[i][j][0]
             for i in self.LESDictionary.keys()
             for j in self.LESDictionary[i].keys()})

        LESDicPoint = pd.DataFrame.from_dict({('PV',jpar): self.LESPointDictionary['PV'][0,jind]
                for jind, jpar in enumerate('x y z'.split())})

        FinalFrame = pd.concat([LESDicPoint, DatFrameLES],axis=1)

        stringTurb = 'uu uv uw vv vw ww'.split()
        for turbparam in stringTurb:
            FinalFrame['RS_' + turbparam] = FinalFrame['MVI'][turbparam] - FinalFrame['IMV'][turbparam]
        if detail is False:
            FinalFrame.drop(['PV','IMV','MVI'],axis=1,inplace=True)
        return FinalFrame

    def save_obj(self,name):
        with open("C:\\Users\\Alvaro\\Desktop\\Alvaro\\Imperial College\\ME4\FYP\\Diction\\" + name + '.pkl', 'wb') as f:
            pickle.dump(self, f, pickle.HIGHEST_PROTOCOL)

    def load_obj(name):
        with open("C:\\Users\\Alvaro\\Desktop\\Alvaro\\Imperial College\\ME4\FYP\\Diction\\" + name + '.pkl', 'rb') as f:
            return pickle.load(f)

DataSetMult = 1
DataSize = 64*DataSetMult
ReductionFactor = 16*DataSetMult
windowsDataPath = Path("C:\\Users\\Alvaro\\Desktop\\Alvaro\\Imperial College\\ME4\\FYP\\Data\\turb3d_" + str(DataSize) + ".dat")
FileToOpen = Path(windowsDataPath)
ProcessData = DataProcessingClass(DataPath=FileToOpen,GridSize=DataSize,Reduction=ReductionFactor)
ProcessData.Create_Data_Structure()
ProcessData.LESDiction()
ProcessData.GaussianCubeCreate()
ProcessData.Coarsen()
DataFrame = ProcessData.CreateDataFrame(False)
# ProcessData.save_obj("LESDict_{}_{}".format(ProcessData.GridSize,ProcessData.HaloSize))
# DataFrame.to_csv(path_or_buf="C:\\Users\\Alvaro\\Desktop\\Alvaro\\Imperial College\\ME4\\FYP\\Data\\DFrame128_2.csv",encoding="utf-8")

def load_obj(name):
    with open("C:\\Users\\Alvaro\\Desktop\\Alvaro\\Imperial College\\ME4\FYP\\Diction\\" + name + '.pkl', 'rb') as f:
        return pickle.load(f)
# ProcessData = load_obj("LESDict_128_4")

C:\Users\USER\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\USER\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\USER\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Alvaro\\Desktop\\Alvaro\\Imperial College\\ME4\\FYP\\Data\\turb3d_64.dat'

In [ ]:
# In[1110]:


# save_obj(ProcessData,"LESDict_{}_{}".format(DataSize,ProcessData.HaloSize))

In [ ]:
get_ipython().run_line_magic('matplotlib', 'qt')

In [ ]:
# DataFrame.to_csv(path_or_buf="C:\\Users\\Alvaro\\Desktop\\Alvaro\\Imperial College\\ME4\\FYP\\Data\\DFrame256.csv",encoding="utf-8")
## When Reading Data From Excel...
# DatUP = pd.read_csv("C:\\Users\\Alvaro\\Desktop\\Alvaro\\Imperial College\\ME4\\FYP\\Data\\Dframe256.csv",encoding="utf-8",delimiter=';')
# DataFrame = DatUP['u v w ux uy uz vx vy vz wx wy wz uu uv uw vv vw ww'.split()].copy()

In [ ]:
ProcessData = load_obj("LESDict_256_4")
DataFrame = ProcessData.CreateDataFrame(False)
DataFrame.columns = DataFrame.columns.droplevel()
DataFrame.columns = ['u', 'v', 'w', 'ux', 'uy', 'uz', 'vx', 'vy', 'vz', 'wx', 'wy', 'wz', 'uu',
      'uv', 'uw', 'vv', 'vw', 'ww']
DataFrame

In [ ]:
import tensorflow as tf
import seaborn as sns

In [ ]:
learning_rate = 0.01
batch_size = 2048
n_inputs = 3 + 9
n_hidden_1 = 24
n_hidden_2 = 12
n_hidden_3 = 24
n_data_points = (ProcessData.Reduction)**3
n_classes = 6
frac = 0.8

In [ ]:
def TestTrainExtract(DataFrame,frac):
    TrainingDF = DataFrame.sample(frac=frac,random_state=1)
    TestDF = DataFrame.drop(index = TrainingDF.index.values,axis = 1)
    return TrainingDF, TestDF

def norm(x):
    return (x - TrainStats['mean']) / TrainStats['std']

def SplitTestEval(TrainFrame):
    test_x, test_y = np.split(TrainFrame,[n_inputs],axis=1)
    return test_x, test_y

def MLPerceptron(x, weights, biases):

    '''
    x: Placeholder for Data Input
    Weights : Dicts of Weights
    Biases: Dicts of Biases
    '''

    # First Layer
    layer_1 = tf.add(tf.matmul(x,weights['w1']),biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)

    # Second layer
    layer_2 = tf.add(tf.matmul(layer_1,weights['w2']),biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)

    # Third Layer
    layer_3 = tf.add(tf.matmul(layer_2,weights['w3']),biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)

    # Output Layer
    OutputLayer = tf.matmul(layer_3,weights['out']) + biases['out']

    return OutputLayer

def NextBatch(BatchSize, BatchIter, RemainingData):
    Batch = RemainingData.sample(n = BatchSize,random_state=1)
    RemainingData.drop(index = Batch.index.values,axis = 1, inplace = True)
    Batch_x, Batch_y = np.split(Batch,[n_inputs],axis=1)
    return Batch_x, Batch_y

def NextBatchNormalised(BatchSize, BatchIter, RemainingData):
    Batch = RemainingData.sample(n = BatchSize,random_state=1)
    RemainingData.drop(index = Batch.index.values,axis = 1, inplace = True)
    return Batch

#MAKE THE TRAIN AND TEST DATA
Train, Test = TestTrainExtract(DataFrame, frac)

test_x, test_y = SplitTestEval(Test)
train_x, train_y = SplitTestEval(Train)

TrainStats = train_x.describe().transpose()

NormedTest_x = norm(test_x)
NormedTrain_x = norm(train_x)

# sns.pairplot(Train[["u", "v","w"]], diag_kind="kde")

#CREATE THE WEIGHTS AND BIASES DICTIONARIES
weights = {
    'w1':tf.Variable(tf.random_normal([n_inputs,n_hidden_1])),
    'w2':tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    'w3':tf.Variable(tf.random_normal([n_hidden_2,n_hidden_3])),
    'out':tf.Variable(tf.random_normal([n_hidden_3,n_classes]))
}

biases = {
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    'b3':tf.Variable(tf.random_normal([n_hidden_3])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

x = tf.placeholder(tf.float32,shape = [None,n_inputs])
y = tf.placeholder(tf.float32,shape = [None,n_classes])

#ASSIGN THE OPISMISIERS AND COST FUNCTION
Prediction = MLPerceptron(x, weights, biases)
Cost = tf.reduce_mean(tf.square(Prediction - y))
Optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(Cost)
#Optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(Cost)

In [ ]:
#with tf.Session() as sess:
tf.InteractiveSession.close(sess)
sess = tf.InteractiveSession()
init = tf.initialize_all_variables()
sess.run(init)
avg_cost_old = 100000
cost_arr = []
epochs = 10

for epoch in range(epochs):
    avg_cost = 0.0
    total_batch_number = int(n_data_points*frac/batch_size)
    RemainingData_x = NormedTrain_x.copy()
    RemainingData_y = train_y.copy()

    for BatchIter in range (total_batch_number):
        Batch_x = NextBatchNormalised(batch_size, BatchIter, RemainingData_x)
        Batch_y = NextBatchNormalised(batch_size, BatchIter, RemainingData_y)
        _,c = sess.run([Optimizer,Cost],feed_dict={x:Batch_x,y:Batch_y})
        avg_cost += c/total_batch_number

    print("Epoch: {}, cost: {}".format(epoch+1, avg_cost))
    cost_arr.append(avg_cost)
    if (avg_cost_old > avg_cost):
        avg_cost_old = avg_cost
    else:
        break

print("Complete Model {} Epochs of training".format(epoch+1))

In [ ]:
plt.plot(np.arange(1,epochs+1),cost_arr,label='Cost',linewidth=2)
plt.xlim([1,epochs])
plt.xlabel("Epoch")
plt.ylabel("MSE Cost")

In [ ]:
preds = Prediction.eval(feed_dict={x:NormedTest_x})
(preds-test_y).describe()

In [ ]:
plt.figure()
plt.rcParams.update({'font.size':10})
summ_dict = {}
for index, param in enumerate('uu uv uw vv vw ww'.split()):
    summ_dict[param] = np.corrcoef(test_y[param],preds[:,index])[0,1]
    plt.subplot(2,3,index + 1)
    sns.scatterplot(test_y[param],preds[:,index],label="SGS ({}), Corr. {:.2f}".format(param,summ_dict[param]))
    plt.legend()
    if index > 2:
        plt.xlabel('Test Value',)
    if index == 0 or index == 3:
        plt.ylabel('Predicted Value')

In [ ]:
summary = pd.DataFrame.from_dict(data=summ_dict,orient='index')
summary.columns = ['Correlation']
summary['Correlation']

In [ ]:
sns.scatterplot(test_y['ww'],preds[:,5])
plt.plot(np.arange(0,60),np.arange(0,60),'r')
err = (test_y['vv'] - preds[:,3])
sns.kdeplot(preds[:,0],test_y['uu'])

In [ ]:
tf.InteractiveSession.close(sess)